Единственная задача: обобщить написанный на семинаре код и сымитировать работу одного большого отдела ABBYY, в котором есть три маленьких подотдела с лингвистами, программистами и комплингом. То есть, что у нас должно быть реализовано:

- родительский класс "работник"
- базовые классы "лингвист", "программист" и "компьютерный лингвист"
- у всех методы work
- классы "босс_лингвист", "босс_программист" и "босс_кл", которые могут наследовать (с подмешиванием) от общего класса "босс"
- у боссов в атрибутах сидят их подчиненные
- босс подотдела получает квесты от менеджера главного отдела и принуждает сотрудников работать
- в главном отделе есть метод для выдачи квестов
- соответственно, используем как наследование, так и композицию с делегированием

In [57]:
class Employee:
    '''Родительский класс для всех работников'''
    def __init__(self, name, surname, salary_coef):
        self.name = name
        self.surname = surname
        self.salary = 350 * salary_coef
        self.bankaccount = 0
    
    def work(self):
        raise NotImplementedError


class Linguist(Employee):
    '''Класс для лингвистов'''
    def __init__(self, name, surname, salary_coef):
        Employee.__init__(self, name, surname, salary_coef)
        self.publications = []

    def work(self, hours, name_of_publ):
        print(f'{self.name} {self.surname} работает...')
        self.bankaccount += self.salary * hours
        self.publications.append(name_of_publ)
        print(f'{self.name} {self.surname} опубликовал: {name_of_publ}')


class Programmer(Employee):
    '''Класс для программистов'''
    def __init__(self, name, surname, salary_coef):
        Employee.__init__(self, name, surname, salary_coef)
        self.projects = []

    def work(self, hours, project_name):
        print(f'{self.name} {self.surname} пишет код...')
        self.bankaccount += self.salary * hours
        self.projects.append(project_name)
        print(f'{self.name} {self.surname} завершил проект: {project_name}')


class ComputationalLinguist(Employee):
    '''Класс для компьютерных лингвистов'''
    def __init__(self, name, surname, salary_coef):
        Employee.__init__(self, name, surname, salary_coef)
        self.projects = []

    def work(self, hours, project_name):
        print(f'{self.name} {self.surname} работает...')
        self.bankaccount += self.salary * hours
        self.projects.append(project_name)
        print(f'{self.name} {self.surname} закончил работу над проектом: {project_name}')


class Boss:
    '''Родительский класс для начальников'''
    def __init__(self, name, surname, salary_coef):
        self.name = name
        self.surname = surname
        self.salary = 500 * salary_coef
        self.bankaccount = 0
        self.subordinates = []

    def add_subordinate(self, subordinate):
        self.subordinates.append(subordinate)

    def assign_task(self, task, hours, project):
        print(f'{self.name} {self.surname} назначает задачу: {task}')
        for subordinate in self.subordinates:
            subordinate.work(hours, project)


class BossLinguist(Boss, Linguist):
    '''Наследуемся одновременно от босса и лингвиста'''
    def __init__(self, name, surname, salary_coef):
        Linguist.__init__(self, name, surname, salary_coef)
        Boss.__init__(self, name, surname, salary_coef)


class BossProgrammer(Boss, Programmer):
    '''Наследуемся одновременно от босса и программиста'''
    def __init__(self, name, surname, salary_coef):
        Programmer.__init__(self, name, surname, salary_coef)
        Boss.__init__(self, name, surname, salary_coef)


class BossCL(Boss, ComputationalLinguist):
    '''Наследуемся одновременно от босса и компьютерного лингвиста'''
    def __init__(self, name, surname, salary_coef):
        ComputationalLinguist.__init__(self, name, surname, salary_coef)
        Boss.__init__(self, name, surname, salary_coef)


class MainDepartment:
    '''Класс для главного отдела'''
    def __init__(self, linguist_boss, programmer_boss, compling_boss):
        self.departments = {
            'ling': linguist_boss,
            'prog': programmer_boss,
            'cl': compling_boss
        }

    def assign_department_task(self, department, task, hours, project):
        '''Назначает задачу определенному подотделу'''
        boss = self.departments.get(department)
        if boss:
            boss.assign_task(task, hours, project)
        else:
            print('Неверный отдел')

In [58]:
# сотрудники
linguist1 = Linguist('Иван', 'Иванов', 1.2)
linguist2 = Linguist('Анна', 'Петрова', 1.3)

programmer1 = Programmer('Петр', 'Петров', 1.5)
programmer2 = Programmer('Мария', 'Иванова', 1.5)

cl1 = ComputationalLinguist('Сергей', 'Соколов', 1.4)
cl2 = ComputationalLinguist('Елена', 'Сергеева', 1.4)

# начальники и их подчиненные
boss_ling = BossLinguist('Босс', 'Лингвистов', 2.0)
boss_ling.add_subordinate(linguist1)
boss_ling.add_subordinate(linguist2)

boss_prog = BossProgrammer('Босс', 'Программистов', 2.3)
boss_prog.add_subordinate(programmer1)
boss_prog.add_subordinate(programmer2)

boss_cl = BossCL('Босс', 'КЛ', 2.1)
boss_cl.add_subordinate(cl1)
boss_cl.add_subordinate(cl2)

# главный отдел
main_department = MainDepartment(boss_ling, boss_prog, boss_cl)

# назначение задач
main_department.assign_department_task('ling', 'Исследование языковых данных', 8, 'Статья 1')
main_department.assign_department_task('prog', 'Разработка программного обеспечения', 8, 'Программа 2')
main_department.assign_department_task('cl', 'Разработка алгоритмов обработки текстов', 8, 'Модель 3')

# босс лингвистов работает
boss_ling.work(5, 'Статья 6')

# проверка баланса
print(f'{linguist1.name} {linguist1.surname} имеет на счету: {linguist1.bankaccount}')
print(f'{programmer1.name} {programmer1.surname} имеет на счету: {programmer1.bankaccount}')
print(f'{cl1.name} {cl1.surname} имеет на счету: {cl1.bankaccount}')
print(f'{boss_ling.name} {boss_ling.surname} имеет на счету: {boss_ling.bankaccount}')

Босс Лингвистов назначает задачу: Исследование языковых данных
Иван Иванов работает...
Иван Иванов опубликовал: Статья 1
Анна Петрова работает...
Анна Петрова опубликовал: Статья 1
Босс Программистов назначает задачу: Разработка программного обеспечения
Петр Петров пишет код...
Петр Петров завершил проект: Программа 2
Мария Иванова пишет код...
Мария Иванова завершил проект: Программа 2
Босс КЛ назначает задачу: Разработка алгоритмов обработки текстов
Сергей Соколов работает...
Сергей Соколов закончил работу над проектом: Модель 3
Елена Сергеева работает...
Елена Сергеева закончил работу над проектом: Модель 3
Босс Лингвистов работает...
Босс Лингвистов опубликовал: Статья 6
Иван Иванов имеет на счету: 3360.0
Петр Петров имеет на счету: 4200.0
Сергей Соколов имеет на счету: 3919.9999999999995
Босс Лингвистов имеет на счету: 5000.0
